# Importing Libraries:

In [15]:
from re import A
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\PANCHAYAT  SAMITI
[nltk_data]     01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Load the data

with open('intents.json', 'r') as f:
    intents = json.load(f)

# Data Preprocessing: (Defining Functions)

In [17]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):

    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0

    return bag

# Neural Network: (Creating the Model)

In [18]:
# creating the neural network that we'll use to train the data
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

# Load model:

In [19]:
FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# Let's chat ;)

In [20]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state) # loading the learnt parametres
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0]) # converting to just one row since we have a single sentence
    X = torch.from_numpy(X).to(device) # converting to torch tensor

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    # during training, crossentropy loss automatically applies softmax.
    # to get maximum probability as the prediction; we manually apply the softmax function below
    
    
    probs = torch.softmax(output, dim=1) 
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Sam: Hey :-)
You: what do you sell
Sam: We sell coffee and tea
You: what is the delivery time
Sam: Hello, thanks for visiting
You: when can I get the delivery
Sam: Delivery takes 2-4 days
You: okay thanks
Sam: My pleasure
You: do you take cash for payment
Sam: We accept most major credit cards, and Paypal
You: Thanks again
Sam: Happy to help!
You: okay goodbye
Sam: Bye! Come back again soon.
You: quit
